In [32]:
#In this notebook we will pre-process the data frame to make it ready for the model

import pandas as pd
import numpy as np

from datetime import datetime

In [33]:
df = pd.read_csv('../data/interim/Steam_2024_bestRevenue_1500.csv')
df

,name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
0,WWE 2K24,07-03-2024,165301,99.99,8055097.0,42.365140,71,AAA,2K,Visual Concepts,2315690
1,EARTH DEFENSE FORCE 6,25-07-2024,159806,59.99,7882151.0,29.651061,57,Indie,D3PUBLISHER,SANDLOT,2291060
2,Sins of a Solar Empire II,15-08-2024,214192,49.99,7815247.0,12.452593,88,Indie,Stardock Entertainment,"Ironclad Games Corporation,Stardock Entertainment",1575940
3,Legend of Mortal,14-06-2024,440998,19.99,7756399.0,24.797817,76,Indie,"Paras Games,Obb Studio Inc.",Obb Studio Inc.,1859910
4,Shin Megami Tensei V: Vengeance,13-06-2024,141306,59.99,7629252.0,34.258496,96,AA,SEGA,ATLUS,1875830
...,...,...,...,...,...,...,...,...,...,...,...
1495,Infection Free Zone,11-04-2024,452146,24.99,8739530.0,14.494432,73,Indie,Games Operators,Jutsu Games,1465460
1496,Dark and Darker,07-06-2024,2640903,0.00,8706135.0,31.974027,74,Indie,IRONMACE,IRONMACE,2016590
1497,Songs of Conquest,20-05-2024,501474,34.99,8641459.0,11.891152,88,AA,Coffee Stain Publishing,Lavapotion,867210
1498,GUNDAM BREAKER 4,28-08-2024,156339,59.99,8440898.0,14.733899,81,AA,Bandai Namco Entertainment Inc.,"CRAFTS ＆ MEISTER Co., Ltd",1672500


In [34]:
df['publishers'].value_counts()

publishers
Kagura Games                     17
Electronic Arts                  16
072 Project                      14
Ubisoft                          13
Mango Party,Mango Party News     11
                                 ..
BLAMCAM Interactive               1
Soda Game Studio                  1
Frogstorm                         1
Significant Steak                 1
Red Nexus Games Inc.,IndieArk     1
Name: count, Length: 1131, dtype: int64

In [35]:
#now we will shrink the number after . in avgPlaytime column. It will be look like 21.4, 21.5, 21.6 etc

df['avgPlaytime'] = df['avgPlaytime'].apply(lambda x: round(x, 1))

In [36]:
df.head()

,name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
0,WWE 2K24,07-03-2024,165301,99.99,8055097.0,42.4,71,AAA,2K,Visual Concepts,2315690
1,EARTH DEFENSE FORCE 6,25-07-2024,159806,59.99,7882151.0,29.7,57,Indie,D3PUBLISHER,SANDLOT,2291060
2,Sins of a Solar Empire II,15-08-2024,214192,49.99,7815247.0,12.5,88,Indie,Stardock Entertainment,"Ironclad Games Corporation,Stardock Entertainment",1575940
3,Legend of Mortal,14-06-2024,440998,19.99,7756399.0,24.8,76,Indie,"Paras Games,Obb Studio Inc.",Obb Studio Inc.,1859910
4,Shin Megami Tensei V: Vengeance,13-06-2024,141306,59.99,7629252.0,34.3,96,AA,SEGA,ATLUS,1875830


In [37]:
#now I would like to see publishers total revenue and add a new column that may repeat the publisher name for each row
#the new column name will be total_revenue and it will be the sum of revenue for each publisher

df['total_revenue'] = df.groupby('publishers')['revenue'].transform('sum')

In [38]:
#now we are adding another column reletad to total revenue. It will be the percentage of game revenue for each publisher
#the new column name will be revenue_percentage

df['revenue_percentage'] = (df['revenue'] / df['total_revenue']) * 100

In [39]:
#again we will shrink the number after . in revenue_percentage column. It will be look like 21.42, 21.53, 21.46 etc
df['revenue_percentage'] = df['revenue_percentage'].apply(lambda x: round(x, 2))

In [40]:
#now we will rename all columns to make them more readable

df.rename(columns={
    'name':'game', 
    'releaseDate':'release_date', 
    'copiesSold':'copies_sold', 
    'price':'price', 
    'avgPlaytime':'avg_playtime', 
    'reviewScore':'review_score',
    'publisherClass':'publisher_class',
    'revenue':'revenue',
    'publishers':'publisher',
    'steamId':'steam_id',
    'total_revenue':'total_revenue',
    'revenue_percentage':'revenue_percentage'
    }, inplace=True)

In [41]:
#this data is created in 08-september-2024. I want to add a new column that shows how long does the game exist in the market
#first we need to convert release_date column to datetime format
#and then we will subtract release_date from 08-09-2024

df['release_date'] = pd.to_datetime(df['release_date'], format="%d-%m-%Y")




In [42]:
# now creating days_in_market column by calculating the difference between current date and release date

# Ensure 'release_date' is in datetime format (if not already)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Set the date to 09-September-2024
current_date = datetime(2024, 9, 9)

# Calculate the days in market
df['days_in_market'] = (current_date - df['release_date']).dt.days

# Replace any negative values with zero
df['days_in_market'] = df['days_in_market'].apply(lambda x: max(x, 0))

# Display the updated DataFrame
df[['game', 'release_date', 'days_in_market']].head()


,game,release_date,days_in_market
0,WWE 2K24,2024-03-07,186
1,EARTH DEFENSE FORCE 6,2024-07-25,46
2,Sins of a Solar Empire II,2024-08-15,25
3,Legend of Mortal,2024-06-14,87
4,Shin Megami Tensei V: Vengeance,2024-06-13,88


In [43]:
df

,game,release_date,copies_sold,price,revenue,avg_playtime,review_score,publisher_class,publisher,developers,steam_id,total_revenue,revenue_percentage,days_in_market
0,WWE 2K24,2024-03-07,165301,99.99,8055097.0,42.4,71,AAA,2K,Visual Concepts,2315690,18878021.0,42.67,186
1,EARTH DEFENSE FORCE 6,2024-07-25,159806,59.99,7882151.0,29.7,57,Indie,D3PUBLISHER,SANDLOT,2291060,7945034.0,99.21,46
2,Sins of a Solar Empire II,2024-08-15,214192,49.99,7815247.0,12.5,88,Indie,Stardock Entertainment,"Ironclad Games Corporation,Stardock Entertainment",1575940,7922689.0,98.64,25
3,Legend of Mortal,2024-06-14,440998,19.99,7756399.0,24.8,76,Indie,"Paras Games,Obb Studio Inc.",Obb Studio Inc.,1859910,7756399.0,100.00,87
4,Shin Megami Tensei V: Vengeance,2024-06-13,141306,59.99,7629252.0,34.3,96,AA,SEGA,ATLUS,1875830,63528353.0,12.01,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Infection Free Zone,2024-04-11,452146,24.99,8739530.0,14.5,73,Indie,Games Operators,Jutsu Games,1465460,8739530.0,100.00,151
1496,Dark and Darker,2024-06-07,2640903,0.00,8706135.0,32.0,74,Indie,IRONMACE,IRONMACE,2016590,8706135.0,100.00,94
1497,Songs of Conquest,2024-05-20,501474,34.99,8641459.0,11.9,88,AA,Coffee Stain Publishing,Lavapotion,867210,13947324.0,61.96,112
1498,GUNDAM BREAKER 4,2024-08-28,156339,59.99,8440898.0,14.7,81,AA,Bandai Namco Entertainment Inc.,"CRAFTS ＆ MEISTER Co., Ltd",1672500,12486291.0,67.60,12


In [44]:
#now we will calculate average_price_per_hour for each game by calculating price / avg_playtime

df['average_price_per_hour'] = df['price'] / df['avg_playtime']
df['average_price_per_hour'] = df['average_price_per_hour'].apply(lambda x: round(x, 2))

In [45]:
df

,game,release_date,copies_sold,price,revenue,avg_playtime,review_score,publisher_class,publisher,developers,steam_id,total_revenue,revenue_percentage,days_in_market,average_price_per_hour
0,WWE 2K24,2024-03-07,165301,99.99,8055097.0,42.4,71,AAA,2K,Visual Concepts,2315690,18878021.0,42.67,186,2.36
1,EARTH DEFENSE FORCE 6,2024-07-25,159806,59.99,7882151.0,29.7,57,Indie,D3PUBLISHER,SANDLOT,2291060,7945034.0,99.21,46,2.02
2,Sins of a Solar Empire II,2024-08-15,214192,49.99,7815247.0,12.5,88,Indie,Stardock Entertainment,"Ironclad Games Corporation,Stardock Entertainment",1575940,7922689.0,98.64,25,4.00
3,Legend of Mortal,2024-06-14,440998,19.99,7756399.0,24.8,76,Indie,"Paras Games,Obb Studio Inc.",Obb Studio Inc.,1859910,7756399.0,100.00,87,0.81
4,Shin Megami Tensei V: Vengeance,2024-06-13,141306,59.99,7629252.0,34.3,96,AA,SEGA,ATLUS,1875830,63528353.0,12.01,88,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,Infection Free Zone,2024-04-11,452146,24.99,8739530.0,14.5,73,Indie,Games Operators,Jutsu Games,1465460,8739530.0,100.00,151,1.72
1496,Dark and Darker,2024-06-07,2640903,0.00,8706135.0,32.0,74,Indie,IRONMACE,IRONMACE,2016590,8706135.0,100.00,94,0.00
1497,Songs of Conquest,2024-05-20,501474,34.99,8641459.0,11.9,88,AA,Coffee Stain Publishing,Lavapotion,867210,13947324.0,61.96,112,2.94
1498,GUNDAM BREAKER 4,2024-08-28,156339,59.99,8440898.0,14.7,81,AA,Bandai Namco Entertainment Inc.,"CRAFTS ＆ MEISTER Co., Ltd",1672500,12486291.0,67.60,12,4.08


In [46]:
#now we prepared our data frame for the model. We will save it as a csv file

df.to_csv('../data/processed/Steam_2024_bestRevenue_1500_02.csv', index=False)
